In [89]:
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print("RuntimeError")
        print(e)


# Ch10. 자연어 처리를 위한 임베딩

임베딩(embedding)은 사람이 사용하는 언어(자연어)를 컴퓨터가 이해할 수 있는 언어(숫자) 형태인 벡터(vector)로 변환한 결과 혹은 일련의 과정을 의미합니다.

In [4]:
# 임베딩
import pandas as pd
class2 = pd.read_csv('./data/class2.csv')
print(class2)

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
2           2  mdb002      A  CIRA      I    NaN    NaN    NaN
3           3  mdb003      C  CIRC      B    NaN    NaN    NaN
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


array([2, 2, 1, 0, 1, 0])

In [8]:
train_x_test = onehot_encoder.fit_transform(class2)
train_x_test

<6x36 sparse matrix of type '<class 'numpy.float64'>'
	with 48 stored elements in Compressed Sparse Row format>

# 횟수 기반 임베딩

## - 카운터 벡터 CountVectorizer()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]
vect = CountVectorizer()   # 객체 생성
vect.fit(corpus)        # 모델 동작
vect.vocabulary_      

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [18]:
# 변형 처리 한 결과를 배열로
vect.transform(['you will never get any chance.']).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [19]:
# 불용어 제거 및 가운터 벡터 확인
# stop_words 옵션에 불용어를 지정하여 제외..
vect = CountVectorizer(stop_words=['and','is','please','this']).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

In [20]:
# 변형 처리 한 결과를 배열로
vect.transform(['you will never get any chance.']).toarray()

array([[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]])

# TF-IDF
### TF- 문서내에 특정 단어의 출현 빈도
### IDF - 특정 단어가 전체 문서 중 출현한 빈도를 Inverse(역수)한 내용
###### 사용하는 상황
- 키워드 검색을 기반으로 하는 검색 엔진
- 중요 키워드 분석
- 검색 엔진에서 검색 결과의 순위를 결정
- TfidfVectorizer()를 사용함

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
doc = ['I like machine learning', 'I love deep learning', 'I run everyday']
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)          # t는 특정단어
print("유사도를 위한", str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]),
      "행렬을 만들었습니다.")
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


# 예측 기반 임베딩
신경망 구조 혹은 모델을 이용하여 특정 문맥에서 어떤 단어가 나올지 예측하면서 단어를 벡터로 만드는 방식
- Word2Vector가 있음
- CBOW(Continious Bag Of Words)
- skip-gram
- 패스트 텍스트(FastText)

In [23]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore') # 경고 무시
import gensim
from gensim.models import Word2Vec

# 샘플 데이터로 peter.txt(피터팬)
sample = open('./data/peter.txt', 'r', encoding='utf-8')
s = sample.read()
s
f = s.replace("\n", " ") # 줄바꿈 "\n" 을 공백 " "으로 변환
data = []

for i in sent_tokenize(f): # 로딩한 파일의 각 문장마다 반복
    temp = []
    for j in word_tokenize(i): #  문장을 단어로 토큰화
        temp.append(j.lower())  # 토큰환 된 단어를 소문자로 변환하여 temp에 저장
    data.append(temp)
data

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

In [ ]:
# CBOW(Continious Bag Of Words)
# 단어를 여러개 나열한 후 이와 관련된 단어를 추정하는 방식
model1 = Word2Vec(data, min_count=1, vector_size=100, window=5, sg=0) # sg값이 0인경우, CBOW
print("Cosine similarity between 'peter','wendy' - CBOW : ",
      model1.wv.similarity('peter', 'wendy'))   # 책에 있는 함수와 유사한 동작하는 함수

Cosine similarity between 'peter','wendy' - CBOW :  0.074393824


In [28]:
# skip-gram

model2 = Word2Vec(data, min_count=1, vector_size=100, window=5, sg=1) # sg값이 1인경우, skip-gram
print("Cosine similarity between 'peter','wendy' - CBOW : ",
      model2.wv.similarity('peter', 'wendy'))   # 책에 있는 함수와 유사한 동작하는 함수

Cosine similarity between 'peter','wendy' - CBOW :  0.40088683


In [30]:
print("Cosine similarity between 'peter','hook' - CBOW : ",
      model2.wv.similarity('peter', 'hook'))   # 책에 있는 함수와 유사한 동작하는 함수

Cosine similarity between 'peter','hook' - CBOW :  0.5201673


In [ ]:
# 패스트 텍스트(FastText)
# 워드투벡터의 단점을 보완하고자 페이스북에서 개발한 임베딩 알고리즘
# 워드투벡터는 워드 임베딩 방식이 분산 표현을 이용하여 단어의 분산 분포가 유사한 단어들에 비슷한 벡터 값을
# 할당하여 표현하는 방식
# 워드투벡터는 사전에 없는 단어에 대해서는 벡터값을 얻을 수 없음
# 자주 사용되지 않는 단어에 대해서는 학습이 불안정
# 패스트 텍스트는 이 단점을 보완하기 위해서 개발된 단어 표현 방식을 사용.
# 패스트텍스트는 노이즈에 강하며, 새로운 단어에 대해서는 형태적 유사성을 고려한 벡터 값을 얻기 때문에
# 자연어 처리 분야에서 많이 사용되고 있음.

In [31]:
# 패스트 테스트
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText('./data/peter.txt', vector_size=4, window=3, min_count=1, epochs=10)

In [33]:
# peter, wendy 유사도 값을 확인
sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

0.4592452


In [34]:
# peter, hook 유사도 값을 확인
sim_score = model.wv.similarity('peter', 'hook')
print(sim_score)

0.043825686


# 사전훈련된 패스트텍스트 모델
https://fasttext.cc/docs/en/pretrained-vectors.html 에서 언어에 맞는 훈련된 모델을 다운받음

In [35]:
from __future__ import print_function
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format('./data/wiki.ko/wiki.ko.vec')

In [39]:
# 사전 훈련된 모델을 이용한 유사도 확인
find_similar_to = '노력'

for similar_word in model_kr.similar_by_word(find_similar_to):
    print("Word : {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word : 노력함, Similarity: 0.80
Word : 노력중, Similarity: 0.75
Word : 노력만, Similarity: 0.72
Word : 노력과, Similarity: 0.71
Word : 노력의, Similarity: 0.69
Word : 노력가, Similarity: 0.69
Word : 노력이나, Similarity: 0.69
Word : 노력없이, Similarity: 0.68
Word : 노력맨, Similarity: 0.68
Word : 노력보다는, Similarity: 0.68


In [40]:
similarities = model_kr.most_similar(positive=['동물','육식동물'], negative=['사람'])
print(similarities)
for sim in similarities:
    print(sim)

[('초식동물', 0.7804122567176819), ('거대동물', 0.7547270655632019), ('육식동물의', 0.7547166347503662), ('유두동물', 0.753511369228363), ('반추동물', 0.7470757961273193), ('독동물', 0.7466291785240173), ('육상동물', 0.746031641960144), ('유즐동물', 0.7450903654098511), ('극피동물', 0.7449344396591187), ('복모동물', 0.742434561252594)]
('초식동물', 0.7804122567176819)
('거대동물', 0.7547270655632019)
('육식동물의', 0.7547166347503662)
('유두동물', 0.753511369228363)
('반추동물', 0.7470757961273193)
('독동물', 0.7466291785240173)
('육상동물', 0.746031641960144)
('유즐동물', 0.7450903654098511)
('극피동물', 0.7449344396591187)
('복모동물', 0.742434561252594)


# 횟수/예측기반 임베딩
앞서 살펴본 횟수 기반과 예측기반의 단점을 보완하기 위한 임베딩 기법
대표적인 모델로 글로브(GloVe)

In [56]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/home/nino/data/glove.6B.100d.txt') # 경로 오류가 발생했을 경우는 절대경로로!
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2ve.txt")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [46]:
%pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [57]:
# bill 관련 단어와 유사한 단어 리스트 보기
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
model.most_similar('bill')

[('legislation', 0.8072139620780945),
 ('proposal', 0.730686366558075),
 ('senate', 0.7142541408538818),
 ('bills', 0.704440176486969),
 ('measure', 0.6958035230636597),
 ('passed', 0.690624475479126),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663140058517456)]

In [58]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [59]:
def analogy(x1,x2, y1):
    result = model.most_similar(positive=[y1,x2], negative=[x1])
    return result[0][0]
analogy('australia', 'beer','france')

'champagne'

In [60]:
analogy('tall', 'tallest', 'long')

'longest'

In [61]:
# 열거된 단어 중 유사성이 가장 떨어지는 단어 반환하는 코드
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


# 트랜스포머 어텐션
- seq2seq

In [4]:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import os
import io
import re
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = re.sub(r"([?.!,¿])", r" \1 ", w) #  특수 문자 제거
    w = re.sub(r'[" "]+', " ", w) # 단어와 그 뒤에 오는 구두점 사이에 공백을 삽입(예 “she is here.” → “she is here . ”)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w) # a-z, A-Z, ., ?, ! 등을 제외하고 모두 공백으로 바꿈
    w = w.rstrip().strip() # 공백 문자 제거
    w = '<start> ' + w + ' <end>' # 문장의 시작 <start>와 종료 <end> 토큰 생성(모델이 예측을 시작하고 종료할 시기를 알 수 있도록 시작과 끝을 지정)
    return w

In [6]:
# 전처리 내용 테스트
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> May I borrow this book ? <end>
b'<start> \xc2\xbf Puedo tomar prestado este libro ? <end>'


In [7]:
# 단어의 악센트 제거. [ENGLISH, SPANISH]형식으로 단어가 반복되게코드 구현
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
 
    return zip(*word_pairs)

def max_length(tensor):
    return max(len(t) for t in tensor)
 
def tokenize(lang):   # 문장의 토큰화
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)
 
  tensor = lang_tokenizer.texts_to_sequences(lang)
 
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
 
  return tensor, lang_tokenizer
 
def load_dataset(path, num_examples=None):    # 입력/출력(english, spanish) 쌍 만들기
    targ_lang, inp_lang = create_dataset(path, num_examples)
 
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
 
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [8]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset('./data/spa.txt', num_examples)
 
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
 
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [7]:
max_length_inp
max_length_targ
len(input_tensor_train) + len(input_tensor_val)
# input_tensor.shape
# target_tensor.shape

30000

In [9]:
# 데이터 분석을 준비하기 앞서 하이퍼파라미터들을 정의
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
 
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
# 데이터 분석을 위한 데이터셋 준비
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# 네트워크 구축 - 인코더 , 디코더
# 인코더 네트워크

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
 
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state
 
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))  # 은닉층 초기화
 
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) 


In [11]:
# 어텐션 구현

class EDAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(EDAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1) #1 텐서의 원하는 위치에 차원 추가
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis))) #2 배치크기 , 시퀸스 최대길이, 1
 
        attention_weights = tf.nn.softmax(score, axis=1) # 어텐션 가주치(attention_weight)의 형태는 (배치크기, 시퀸스 최대 길이, 1)이 됩니다.
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1) # 컨텍스트 벡터(context_vector)의 형태는 (배치크기, 은닉층 크기)입니다.
        return context_vector, attention_weights 
attention_layer = EDAttention(10)

In [12]:

# 디코더 네트워크
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = EDAttention(self.dec_units) #어텐션 적용
 
    def call(self, x, hidden, enc_output):  # 인코더 출력(enc_output)형태는 (배치크기, 시퀸스 최대 길이, 은닉층 크기)입니다.
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)  # 임베딩층을 통과한 후 x의 형태는 (배치크기, 1, 임베딩 차원)입니다.
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)  # 병합된 벡터를 GRU로 보냅니다.
        output = tf.reshape(output, (-1, output.shape[2])) # 출력 형태는 (배치크기x1, 은닉층 크기)입니다..
        x = self.fc(output)
        return x, state, attention_weights
 
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [13]:
# 모델 훈련을 위한 옵티마이저와 손실함수 정의
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
 
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [14]:
# 모델의 저장과 복원을 위한 체크포인트 설정
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [15]:
# 모델을 훈련시키기 위한 함수 정의
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]): # 대상 단어를 입력으로 사용
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output) # 인코더 출력(enc_output)을 디코더로 보냄
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
  batch_loss = (loss / int(targ.shape[1]))  # 손실/오차 계산
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables)) 
  return batch_loss

In [ ]:
# 모델 훈련
EPOCHS = 10
 
for epoch in range(EPOCHS):
  start = time.time()
 
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
 
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
 
    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  if (epoch + 1) % 2 == 0:  # 2 에포크마다 모델을 체크포인트에 저장
    checkpoint.save(file_prefix = checkpoint_prefix)
 
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6988
Epoch 1 Batch 100 Loss 2.1160
Epoch 1 Batch 200 Loss 1.9169
Epoch 1 Batch 300 Loss 1.7131
Epoch 1 Loss 1.9748
Epoch 2 Batch 0 Loss 1.5647
Epoch 2 Batch 100 Loss 1.4435
Epoch 2 Batch 200 Loss 1.2952
Epoch 2 Batch 300 Loss 1.2741
Epoch 2 Loss 1.3348
Epoch 3 Batch 0 Loss 1.1099
Epoch 3 Batch 100 Loss 0.9949
Epoch 3 Batch 200 Loss 0.9340
Epoch 3 Batch 300 Loss 0.8392
Epoch 3 Loss 0.9650
Epoch 4 Batch 0 Loss 0.6550
Epoch 4 Batch 100 Loss 0.7067
Epoch 4 Batch 200 Loss 0.5845
Epoch 4 Batch 300 Loss 0.5948
Epoch 4 Loss 0.6671
Epoch 5 Batch 0 Loss 0.4394


In [1]:
# 모델 평가 함수
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
 
    sentence = preprocess_sentence(sentence)
 
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
 
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))  # 어텐션 가중치
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        dec_input = tf.expand_dims([predicted_id], 0)    # 예측된 ID가 모델에 피드백
 
    return result, sentence, attention_plot

In [2]:
# 어텐션 가중치 시각화를 위한 함수 정의
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [16]:
# 번역을 위한 함수 정의 및 번역 문장 입력 함수
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)
 
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
 
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' ')) #어텐션 가중치 매핑

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))  #1

translate(u'esta es mi vida.')   # 스페인어를 영어로 번역

NameError: name 'np' is not defined